## <b>M5_종합실습</b>

### 1.정형 데이터 분석

- csv 데이터 불러오기 : 환경부 국가미세먼지정보센터_대기배출원조사 결과_20201231.csv

In [ ]:
import pandas as pd

df = pd.read_csv('data_set/환경부 국가미세먼지정보센터_대기배출원조사 결과_20201231.csv', encoding='euc-kr')
df

In [ ]:
data = open('data_set/환경부 국가미세먼지정보센터_대기배출원조사 결과_20201231.csv', 'rb').readline()
data

In [ ]:
import chardet
data = open('data_set/환경부 국가미세먼지정보센터_대기배출원조사 결과_20201231.csv', 'rb').readline()
result = chardet.detect(data)
print(result['encoding'])

- 컬럼 추가하고 계산하기

In [ ]:
df['평균'] = df.mean(axis=1, numeric_only=True)

In [ ]:
df['합계'] = df.sum(axis=1, numeric_only=True)

In [ ]:
df['사업장평균'] = df.iloc[:,2:5].mean(axis=1, numeric_only=True)

In [ ]:
df

- 데이터 프레임을 이미지로 내보내기

In [ ]:
import dataframe_image as dfi
dfi.export(df, 'mean.png', max_rows=-1, max_cols=-1)

In [ ]:
df.to_clipboard()

### 2.웹 데이터 추출

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By

- 대한민국헌법(1장~10장, 부칙) 텍스트 데이터 가져오기 

In [ ]:
url = 'https://www.law.go.kr/LSW/lsInfoP.do?efYd=19880225&lsiSeq=61603#0000'
driver = webdriver.Chrome()
driver.get(url)

In [ ]:
law_text = driver.find_element(By.XPATH, '//*[@id="conScroll"]').text
driver.quit() 


In [ ]:
driver

In [ ]:
law_text[8000:]

### 3.텍스트 분석(빈도 분석)

- 형태소 분석

In [ ]:
from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords

kiwi = Kiwi()
stopwords = Stopwords()

tokens = kiwi.tokenize(law_text, stopwords=stopwords)
tokens[:10]

- 단어만 추출

In [ ]:
token_text = []

for token in tokens:
    token_text.append(token.form)

token_text

- 단어 빈도수 측정

In [ ]:
from collections import Counter

In [ ]:
c = Counter(token_text)
c

In [ ]:
top_c = c.most_common(10)
top_c

In [ ]:
top_c = dict(top_c)

In [ ]:
top_c

- 단어 빈도 시각화(그래프)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import font_manager as fm
import matplotlib as mpl

In [ ]:
plt.style.use('bmh')
%config InlineBackend.figure_format = 'retina'

- 폰트 매니저에서 관리하는 폰트 확인

In [ ]:
set([f.name for f in mpl.font_manager.fontManager.ttflist]) #set 집합

- 폰트 설정

In [ ]:
plt.rc('font', family='Gulim', size=7)

- 세로 막대 그래프 그리기

In [ ]:
# import numpy as np

# x = np.arange(len(top_c))
# x

In [ ]:
plt.figure(figsize=(10, 5))
plt.bar(x, top_c.values(), width=0.5, color='red')
plt.xticks(x, top_c.keys())
plt.show();

In [ ]:
plt.figure(figsize=(10, 5))
plt.bar(top_c.keys(), top_c.values(), width=0.5, color='red')
plt.savefig('law.png')
plt.show();

In [ ]:
plt.savefig('law.png')

### 4.EP 메일 보내기

- EP 로그인 정보 입력(ID, PASS)

In [ ]:
import pyautogui as pg
from selenium.webdriver.common.keys import Keys
import clipboard as cb

In [ ]:
my_id = 'yooleenes'
my_pw = pg.password(title='EP PASS', text='EP PASS를 입력하세요')

- EP 접속

In [ ]:
url = 'http://swpsso.posco.net/idms/U61/jsp/login/login.jsp'

driver = webdriver.Edge()
driver.get(url)

driver.implicitly_wait(10)
driver.maximize_window()

driver.find_element(By.XPATH,'//*[@id="username"]').send_keys(my_id)
driver.find_element(By.XPATH,'//*[@id="password"]').send_keys(my_pw)
driver.find_element(By.XPATH,'//*[@id="loginsubmit"]').click()

pg.sleep(5)
try:
    alert = driver.switch_to.alert
    alert.accept()
except:
    print('기존 로그인 창 없음, 계속 진행해라')

- 메일 화면 열기

In [ ]:
driver.switch_to.window(driver.window_handles[-1])
driver.find_element(By.XPATH, '//*[@id="533982"]/a[1]').click()
pg.sleep(3)

- 메일 쓰기 화면 열기

In [ ]:
driver.switch_to.window(driver.window_handles[-1])
driver.find_element(By.XPATH, '//*[@id="Lnb"]/div[1]/a').click()
pg.sleep(3)

- 메일 정보, 본문 내용 입력

In [ ]:
mail_add = 'yooleenes@poscohrd.com'
mail_sub = '테스트 메일입니다.'
main_con1 = '데이터 분석 결과 보내 드립니다'
main_con2 = '- 환경부 국가미세먼지정보센터 대기배출원 조사 결과입니다.'
main_con3 = '- 대한민국 헌법 키워드 분석 결과입니다.'

- 메일 주소 입력

In [ ]:
driver.switch_to.window(driver.window_handles[-1])
driver.find_element(By.XPATH, '//*[@id="token-input-send_to"]').send_keys(mail_add)
driver.find_element(By.XPATH, '//*[@id="token-input-send_to"]').send_keys(Keys.ENTER)
pg.sleep(3)

- 메일 제목 입력

In [ ]:
driver.find_element(By.XPATH, '//*[@id="write_send_info"]/table[2]/tbody/tr[1]/td/input').send_keys(main_con1)

- 메일 본문 입력(기본 문구)

In [ ]:
driver.switch_to.frame('dext_frame_WEC')
driver.switch_to.frame('dext5_design_WEC')

# driver.find_element(By.XPATH, '//*[@id="dext_body"]/p[1]').send_keys(Keys.ENTER)
driver.find_element(By.XPATH, '//*[@id="dext_body"]/p[1]').send_keys(main_con1)
driver.find_element(By.XPATH, '//*[@id="dext_body"]/p[1]').send_keys(Keys.ENTER)
driver.find_element(By.XPATH, '//*[@id="dext_body"]/p[1]').send_keys(Keys.ENTER)



- 메일 본문 입력(정형 데이터)

In [ ]:
driver.find_element(By.XPATH, '//*[@id="dext_body"]/p[1]').send_keys(main_con2)

- 이미지 복사(정형 데이터 테이블)

In [ ]:
import pywinauto
pg.hotkey('win', 'r')
cb.copy('D:\작업방\Level2_office\mean.png')
pg.sleep(0.5)
pg.hotkey('ctrl', 'v')
pg.sleep(0.5)
pg.press('enter')
pg.sleep(10)


# win = pg.getWindowsWithTitle("사진")[0]
# pywinauto.application.Application().connect(handle=win._hWnd).top_window().set_focus()
# pg.sleep(1)
# win.activate()

win = pg.getWindowsWithTitle('사진')[0]
win.activate()

pg.hotkey('ctrl', 'c')

- 이미지 붙여넣기(정형 데이터 테이블)

In [ ]:
win = pg.getWindowsWithTitle('메일쓰기')[0]
win.activate()

pg.press('enter', presses=2)
pg.hotkey('ctrl', 'v')
pg.sleep(1)
pg.press('tab', presses=5, interval=0.2)
pg.write('1200')
pg.press('tab', presses=5, interval=0.2)
pg.press('enter')

pg.press('enter')

- 메일 본문 입력(비정형 데이터)

- 이미지 복사(비정형 데이터 그래프)

- 이미지 붙여넣기(비정형 데이터 그래프)

- 본문 작성 완료

In [ ]:
driver.switch_to.default_content()

- 보내기

In [ ]:
driver.switch_to.window(driver.window_handles[-1])
driver.find_element(By.XPATH,'//*[@id="memo_content"]/div[1]/ul[1]/li[1]/a').click()

pg.sleep(5)

try:
    alert = driver.switch_to.alert
    alert.accept()
except:
    print('메일 발송 확인창 없음')
